Before your interview, write a program that lets two humans play a game of Tic Tac Toe in a terminal. 
The program should let the players take turns to input their moves. 
The program should report the outcome of the game

In [1]:
%matplotlib widget
import itertools
import random  

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.path as mpath
import matplotlib.cm as mcm
import matplotlib.transforms as mtransforms
from matplotlib.collections import PatchCollection
# https://matplotlib.org/3.1.1/gallery/shapes_and_collections/artist_reference.html
import numpy as np

In [50]:
class TicTacToe(object):
    loc = [0, .33, .66]
    inds = dict(zip(loc, [0,1,2]))
    facecolor = "0.99"
    gridcolor = "#11557c"
    edgecolor = '0.5'
    radii = 10 * np.array([0.2, 0.6, 0.8, 0.7, 0.4, 0.5, 0.8])
    colors = [mcm.jet(r/10.) for r in radii]

    def __init__(self, game_type='auto'):
        self.game_type = game_type
        
        self.players = itertools.cycle(['X','O'])
        self.player = next(self.players)
        self.get_symbol = {'X':self.draw_x, 'O':self.draw_o}
        self.board = np.empty(shape=(3,3), dtype='object')
        self.plays = 0
        self.game_over = False
            
        self.fig, self.ax = plt.subplots()
        
        #MPL dev Tom Caswell suggested I use patches
        for (x,y) in itertools.product(self.loc, repeat=2):
             patch = mpatches.Rectangle((x,y), height=.3, width=.3, 
                                        ec=self.edgecolor, fc=self.facecolor,
                                        picker=True)
             self.ax.add_patch(patch)

        self.ax.set_aspect('equal')
        self.ax.set_xlim(0,.96) # set clean borders
        self.ax.set_ylim(0,.96)
        self.ax.set_facecolor(self.gridcolor)
        for spine in self.ax.spines.values():
            spine.set_edgecolor(self.edgecolor)
        self.ax.set_xticks([])
        self.ax.set_yticks([])

        # Connect the click function to the button press event
        # bind pick events to our on_pick function
        self.fig.canvas.mpl_connect('pick_event', self.on_pick)
        plt.show()

    def draw_x(self, x, y):
        width, height = .3, .3
        span = .03
        verts = [(x+width/2-span, y), (x+width/2+span,y), (x+width/2+span,y+height/2-span), 
                 (x+width,y+height/2-span), (x+width, y+height/2+span), (x+width/2+span, y+height/2+span),
                 (x+width/2+span, y+height), (x+width/2-span, y+height), (x+width/2-span, y+height/2+span),
                 (x, y+height/2+span), (x, y+height/2-span), (x+width/2-span, y+height/2-span), 
                 (x+width/2-span, y)]

        codes = [mpath.Path.MOVETO, mpath.Path.LINETO, mpath.Path.LINETO, mpath.Path.LINETO, mpath.Path.LINETO,
             mpath.Path.LINETO, mpath.Path.LINETO, mpath.Path.LINETO, mpath.Path.LINETO,mpath.Path.LINETO, 
             mpath.Path.LINETO, mpath.Path.LINETO, mpath.Path.CLOSEPOLY,]

        path = mpath.Path(verts, codes)
        #https://stackoverflow.com/questions/4285103/matplotlib-rotating-a-patch
        path = path.transformed(mtransforms.Affine2D().rotate_deg_around(x+width/2, y+height/2, 45))
        return [mpatches.PathPatch(path, fc=self.colors[2], ec = self.gridcolor)]
        
    
    def draw_o(self, x, y):
        return [mpatches.Circle((x +.15, y +.15), radius=.13, fc=self.colors[1], ec=self.gridcolor),
                mpatches.Circle((x +.15, y +.15), radius=.07, fc=self.facecolor, ec=self.gridcolor)]
       
    def check_win(self):
        """
        down, across, \ diagonal, /diagonal  
        """
        return ((self.board==self.player).all(axis=0).any() | 
                (self.board==self.player).all(axis=1).any() | 
                (np.diag(self.board)==self.player).all()|
                (np.diag(np.flip(self.board,axis=1))== self.player).all())
         
    def auto_move(self):
        xcoord, ycoord = np.where(self.board==None)
        x, y = random.choice(list(zip(xcoord, ycoord)))
        self.draw_symbol(self.loc[x], self.loc[y])
        self.next_turn()
    
    #https://jakevdp.github.io/blog/2012/12/06/minesweeper-in-matplotlib/
    # Function to be called when mouse is clicked
    # create a function to be bound to pick events: here the event has an
    # attribute `artist` which points to the object which was clicked
    
    def draw_symbol(self, x, y):
        patches = self.get_symbol[self.player](x, y)
        [self.ax.add_patch(p) for p in patches]
        self.fig.canvas.draw()
        self.board[self.inds[x], self.inds[y]] = self.player
        self.plays += 1
        
    
    def next_turn(self):
        if self.plays > 4 and self.check_win():
            self.ax.set_title(f'Player {self.player} won!', fontsize=24)
            self.game_over = True
            self.fig.canvas.draw()
        elif self.plays == 9:
            self.ax.set_title("Draw",fontsize=24)
            self.game_over = True
            self.fig.canvas.draw()
        else:
            self.player = next(self.players)
        return
    
    def on_pick(self, event):  
        xy = event.artist.get_xy()
        x, y = xy[0], xy[1]
        
        if self.game_over or (self.board[self.inds[x], self.inds[y]] is not None):
            return #don't overwrite board 
        
        self.draw_symbol(x, y)
        self.next_turn()
        
        if not self.game_over and self.game_type == 'auto':
            self.auto_move()
        return

In [56]:
TicTacToe('auto')

/home/hannah/miniconda3/envs/mos/lib/python3.7/site-packages/matplotlib/pyplot.py:514: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


FigureCanvasNbAgg()